### Import Libraries

In [3]:
import pandas as pd
import numpy as np

#### Import dataset

For those students not using Turi Create please download the training and testing data csv files.

In [7]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}

house_data = pd.read_csv(filepath_or_buffer= './data/kc_house_data.csv', dtype = dtype_dict)  
train_data = pd.read_csv('./data/kc_house_train_data.csv', dtype = dtype_dict)  
test_data = pd.read_csv('./data/kc_house_test_data.csv', dtype = dtype_dict)  

In [9]:
house_data.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  object 
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  float64
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  float64
 6   sqft_lot       21613 non-null  int32  
 7   floors         21613 non-null  object 
 8   waterfront     21613 non-null  int32  
 9   view           21613 non-null  int32  
 10  condition      21613 non-null  int32  
 11  grade          21613 non-null  int32  
 12  sqft_above     21613 non-null  int32  
 13  sqft_basement  21613 non-null  int32  
 14  yr_built       21613 non-null  int32  
 15  yr_renovated   21613 non-null  int32  
 16  zipcode        21613 non-null  object 
 17  lat            21613 non-null  float64
 18  long  

In [10]:
train_data.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17384 entries, 0 to 17383
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             17384 non-null  object 
 1   date           17384 non-null  object 
 2   price          17384 non-null  float64
 3   bedrooms       17384 non-null  float64
 4   bathrooms      17384 non-null  float64
 5   sqft_living    17384 non-null  float64
 6   sqft_lot       17384 non-null  int32  
 7   floors         17384 non-null  object 
 8   waterfront     17384 non-null  int32  
 9   view           17384 non-null  int32  
 10  condition      17384 non-null  int32  
 11  grade          17384 non-null  int32  
 12  sqft_above     17384 non-null  int32  
 13  sqft_basement  17384 non-null  int32  
 14  yr_built       17384 non-null  int32  
 15  yr_renovated   17384 non-null  int32  
 16  zipcode        17384 non-null  object 
 17  lat            17384 non-null  float64
 18  long  

In [11]:
test_data.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4229 entries, 0 to 4228
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             4229 non-null   object 
 1   date           4229 non-null   object 
 2   price          4229 non-null   float64
 3   bedrooms       4229 non-null   float64
 4   bathrooms      4229 non-null   float64
 5   sqft_living    4229 non-null   float64
 6   sqft_lot       4229 non-null   int32  
 7   floors         4229 non-null   object 
 8   waterfront     4229 non-null   int32  
 9   view           4229 non-null   int32  
 10  condition      4229 non-null   int32  
 11  grade          4229 non-null   int32  
 12  sqft_above     4229 non-null   int32  
 13  sqft_basement  4229 non-null   int32  
 14  yr_built       4229 non-null   int32  
 15  yr_renovated   4229 non-null   int32  
 16  zipcode        4229 non-null   object 
 17  lat            4229 non-null   float64
 18  long    

3. Write a generic function that accepts a column of data (e.g, an SArray) ‘input_feature’ and another column ‘output’ and returns the Simple Linear Regression parameters ‘intercept’ and ‘slope’.

Use the closed form solution from lecture to calculate the slope and intercept. e.g. in python:

In [ ]:
# def simple_linear_regression(input_feature, output):
#     [your code here]
# return(intercept, slope)

4. Use your function to calculate the estimated slope and intercept on the training data to predict ‘price’ given ‘sqft_living’.

save the value of the slope and intercept for later (you might want to call them e.g. squarfeet_slope, and squarefeet_intercept)

In [ ]:
# input_feature = train_data[‘sqft_living’]
# output = train_data[‘price’]

5. Write a function that accepts a column of data ‘input_feature’, the ‘slope’, and the ‘intercept’ you learned, and returns an a column of predictions ‘predicted_output’ for each entry in the input column. e.g. in python:

In [ ]:
# def get_regression_predictions(input_feature, intercept, slope)
#     [your code here]
# return(predicted_output)

6. Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?

7. Write a function that accepts column of data: ‘input_feature’, and ‘output’ and the regression parameters ‘slope’ and ‘intercept’  and outputs the Residual Sum of Squares (RSS). e.g. in python:

In [13]:
# def get_residual_sum_of_squares(input_feature, output, intercept,slope):
#     [your code here]
# return(RSS)

SyntaxError: invalid syntax (<ipython-input-13-bc22c60e06c9>, line 2)

Recall that the RSS is the sum of the squares of the prediction errors (difference between output and prediction). 

8. Quiz Question: According to this function and the slope and intercept from (4) What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?

9. Note that although we estimated the regression slope and intercept in order to predict the output from the input, since this is a simple linear relationship with only two variables we can invert the linear function to estimate the input given the output!

Write a function that accept a column of data:‘output’ and the regression parameters ‘slope’ and ‘intercept’ and outputs the column of data: ‘estimated_input’. Do this by solving the linear function output = intercept + slope*input for the ‘input’ variable (i.e. ‘input’ should be on one side of the equals sign by itself). e.g. in python

In [ ]:
# def inverse_regression_predictions(output, intercept, slope):
#     [your code here]
# return(estimated_input)

10. Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?

11. Instead of using ‘sqft_living’ to estimate prices we could use ‘bedrooms’ (a count of the number of bedrooms in the house) to estimate prices. Using your function from (3) calculate the Simple Linear Regression slope and intercept for estimating price based on bedrooms. Save this slope and intercept for later (you might want to call them e.g. bedroom_slope, bedroom_intercept).

12. Now that we have 2 different models compute the RSS from BOTH models on TEST data.

13. Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case